In [1]:
import sys, os
import pandas as pd
import numpy as np
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(1, '../scripts')
root = os.getcwd()


from TwintDataMiner import TwintDataMiner
from PandasDataCleaner import PandasDataCleaner
from TextBlobSentiment import TextBlobSentiment
from Visualisation import Visualisation
from GMM import GMM


In [2]:
tweetsFile = "../data/1000TweetsPerDay.csv"
tdm = TwintDataMiner()
attributes = ["id", "tweet", "place", "date"]
df = tdm.getPandasDataFrame(attributes, tweetsFile)
df.shape
df.head

<bound method NDFrame.head of                         id                                              tweet  \
0                       id                                              tweet   
1      1212480089155354628  El M*gnici*io es la única opción para Venezuel...   
2      1212479875220566016  plying to @ironorehopper\nInteresting, further...   
3      1212475589325676544  Bienvenidos a los Fabulosos años 20 con @Mierc...   
4      1212473830758899715  Meu Deus, tão inventando nome pra tudo "White ...   
...                    ...                                                ...   
89952  1247297335035641859  Wisconsin Supreme Court, on primary eve, block...   
89953  1247297334741999616  What is becoming of us?    Teen Girls Attack #...   
89954  1247297333743751168  Why does every article about this talks about ...   
89955  1247297333676605440  Recado para senhora corona vírus:\n\nse vc se ...   
89956  1247297333601153024  Schumer names coronavirus czar candidates in p...  

In [3]:
df.drop_duplicates(subset='id')
df.shape

(89957, 4)

In [4]:
pdc = PandasDataCleaner()
def cleandf(df):
    df  = pdc.detectLanguage(df, 'tweet', 'lang')
    df = pdc.cleanTweets(df, 'tweet')
    df = df.loc[df['lang'] == 'en']
    df = df.reset_index(drop=True)
    return df

In [ ]:
vocab = ['death', 'deaths', 'ventilator', 'ventilators', 'crisis', 'scared', 'tragedy', 'struggling']
bowMatrix = np.empty((1, len(vocab)), int)
gmm = GMM()
#generate bag of words for each day and add to matrix
for j in range(1,5):
    for i in range(1,32):
        if(not((j == 1 and i < 10) or (j == 2 and i > 29) or (j ==4 and i > 6))):
            date = "2020-0" + str(j) + "-" + str(i).zfill(2)
            print(date)
            tempdf = df[df.date == date].copy()
            tempdf = tempdf.reset_index(drop=True)

            tempdf.sort_values("id", inplace=True) 
  
            # dropping duplicate values 
            tempdf.drop_duplicates(subset=["tweet"],inplace=True) 
            pdc = PandasDataCleaner()
            tempdf  = pdc.detectLanguage(tempdf, 'tweet', 'lang')
            tempdf = tempdf.loc[tempdf['lang'] == 'en']
            tempdf = pdc.cleanTweets(tempdf, 'tweet')
            tempdf =tempdf[:250]
            
            
            bow = gmm.generateBagOfWords(tempdf, 'tweet', vocab=vocab)
            #get total bag of words for day
            panicRow = np.sum(bow, axis=0)
            bowMatrix = np.append(bowMatrix, np.reshape(panicRow, (1, len(vocab))), axis=0)
            
#delete row created for initialisation          
bowMatrix = np.delete(bowMatrix, (0), axis=0)
print(bowMatrix.shape)
print(bowMatrix)


2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19


In [ ]:
panic = gmm.modelFromBow(bowMatrix)

In [ ]:
print(panic)

In [ ]:
onePanic = panic[:, 0]
print(onePanic)

In [ ]:
plt.plot(onePanic)
plt.show()